<a href="https://colab.research.google.com/github/stemgene/Predict-Stock-Price-by-LSTM-and-Twitter-Analysis-/blob/master/2_nlp_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the code of natrual language processing of Trump's Twitter.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd   
import datetime
import re
import nltk
import gensim
from gensim import corpora, models, similarities
from pprint import pprint

In [3]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
!pip install wordcloud

In [0]:
import wordcloud
w = wordcloud.WordCloud()

In [6]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 2.9MB/s 


In [0]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Data Acquisition

This project focuses on the tweets of President Donald Trump during his presidency, from November 01, 2016 to October 31, 2019.

We exported from online Trump’s tweets archive, and got tweets.json file with 12295 tweets from @realDonaldTrump during his presidency.

In [0]:
data = pd.read_json("/content/drive/My Drive/UR/DSC440_Data_Mining/Project/Data/tweets.json")

In [9]:
data.head()

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
0,Twitter for iPhone,https://t.co/z0I7wBsgTP,2019-10-31 01:26:48+00:00,38879,129556,0.0,1189715310766645248
1,Twitter Media Studio,"""The Democrats have been on this path for 3 ye...",2019-10-30 20:42:43+00:00,20062,63830,0.0,1189643818431406080
2,Twitter for iPhone,Thank you @GLFOP! https://t.co/IpoUY7QPM2,2019-10-30 20:21:54+00:00,11113,38294,0.0,1189638577866657792
3,Twitter for iPhone,“Adam Schiff’s drive to impeach Trump based on...,2019-10-30 19:54:02+00:00,18913,61417,0.0,1189631565279760384
4,Twitter for iPhone,We will be having a big rally for Governor @Ma...,2019-10-30 19:49:44+00:00,13014,42063,0.0,1189630486886395904


In [10]:
data.tail()

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
12290,Twitter for iPhone,So terrible that Crooked didn't report she got...,2016-11-01 14:14:09+00:00,32713,80724,0.0,793456094198759424
12291,Twitter for iPhone,JOIN ME TOMORROW IN FLORIDA!\n\nMIAMI•12pm\nht...,2016-11-01 13:35:47+00:00,8773,17583,0.0,793446442031575040
12292,Twitter for Android,Crooked Hillary should not be allowed to run f...,2016-11-01 12:01:11+00:00,23747,59656,0.0,793422633111023616
12293,Twitter for Android,"Wow, now leading in @ABC /@washingtonpost Poll...",2016-11-01 11:55:47+00:00,16479,39631,0.0,793421274043285504
12294,Twitter for Android,Look at the way Crooked Hillary is handling th...,2016-11-01 10:31:46+00:00,21459,55584,0.0,793400131525677056


In [0]:
# we only interested in text, created_at columns
data = data[['text','created_at']].dropna()

Using created_at column, we label each tweet with a date timestamp:

If the tweet is posted on 12am-4pm from Monday to Friday, we label the tweet the date it posted; 

If the tweet is posted after 4pm from Monday to Thursday, we make the date it posted plus one; 

If the tweet is posted after 4pm on Friday, we make the date plus three; 

If the tweet is posted after 4pm on Saturday, we make the date plus two; 

If the tweet is posted after 4pm on Sunday, we make the date plus one.  

In [12]:
data

,text,created_at
0,https://t.co/z0I7wBsgTP,2019-10-31 01:26:48+00:00
1,"""The Democrats have been on this path for 3 ye...",2019-10-30 20:42:43+00:00
2,Thank you @GLFOP! https://t.co/IpoUY7QPM2,2019-10-30 20:21:54+00:00
3,“Adam Schiff’s drive to impeach Trump based on...,2019-10-30 19:54:02+00:00
4,We will be having a big rally for Governor @Ma...,2019-10-30 19:49:44+00:00
...,...,...
12290,So terrible that Crooked didn't report she got...,2016-11-01 14:14:09+00:00
12291,JOIN ME TOMORROW IN FLORIDA!\n\nMIAMI•12pm\nht...,2016-11-01 13:35:47+00:00
12292,Crooked Hillary should not be allowed to run f...,2016-11-01 12:01:11+00:00
12293,"Wow, now leading in @ABC /@washingtonpost Poll...",2016-11-01 11:55:47+00:00


In [13]:
for i in range(0, 12295):
  timestamp = data['created_at'][i]
  date = timestamp.date()
  if timestamp.weekday() == 6:
    date = date+datetime.timedelta(days = 1)
  elif timestamp.weekday() == 5:
    date = date+datetime.timedelta(days = 2)
  elif timestamp.weekday() == 4 and timestamp.hour >= 16:
    date = date+datetime.timedelta(days = 3)
  elif  timestamp.hour >= 16:
    date = date+datetime.timedelta(days = 1)
  else:
    date = date
  data['created_at'][i] = date

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Then we combine the tweets of the same date timestamp into one document.

In total, we have 775 tweet documents

In [0]:
group = data.groupby(data['created_at']).apply(lambda x:' '.join(x['text']))
#group = data.groupby(data['created_at'])["text"].sum()

In [0]:
raw = pd.DataFrame({'date':group.index, 'text':group.values})

In [0]:
raw

,date,text
0,2016-11-01,So terrible that Crooked didn't report she got...
1,2016-11-02,Praying for the families of the two Iowa polic...
2,2016-11-03,Looking at Air Force One @ MIA. Why is he camp...
3,2016-11-04,"Join me today in Wilmington, Ohio at 4pm: http..."
4,2016-11-07,Thank you for you support Virginia! In ONE DAY...
...,...,...
770,2019-10-25,“Donald J. Trump is an absolutely historic Pre...
771,2019-10-28,The S&amp;P just hit an ALL TIME HIGH. This is...
772,2019-10-29,Nervous Nancy Pelosi is doing everything possi...
773,2019-10-30,....the United States Military. Congress needs...


# Data Cleaning

## Topic Generation Cleaning
we clean the document’s text into bag of words so that we could apply LDA model.

In [0]:
def clean_text(text):
    text  = text.replace('\n'," ")  #replce new line with space
    text = re.sub('-'," ",text)  #seperate "-"connected words, like e-mail
    text = re.sub(r"@[a-zA-Z0-9_]{1,15}", "", text) #delete@username, @[any word, number, _]{length}
    text = re.sub(r"([’']\w{1,2})\b", "", text)  # delete 's or 't, like Mexico's and didn't
    text = re.sub(r"[a-zA-Z]*[:\//\]*[A-Za-z0-9\-_]+\.+[A-Za-z0-9\.\/%&=\?\-_]+", "", text)  #delete urls
    pure_text = ''
    # in case there are still special characters(numbers), we loop the text over to filter them out
    for letter in text:
        if letter.isalpha() or letter ==' ':
            # only keeps alphbet and whitespace
            pure_text += letter
    text = ' '.join(word for word in pure_text.split() if len(word)>1)
    # singleton word left is deleted
    # only left with meaningful words
    return text

In [0]:
docs = raw['text']
docs = docs.apply(lambda x :clean_text(x))

In [0]:
doclist = docs.values

Remove stopwords

In [0]:
stopwords = set(stopwords.words('english'))

In [0]:
stoplist = []
for word in stopwords:
  temp = ''
  for letter in word:
    if letter.isalpha():
      temp += letter
  stoplist.append(temp)
stoplist.extend(['rt', 'amp'])    # 'rt', 'amp' are twitter specific stopwords

In [0]:
texts = [[word for word in doc.lower().split() if word not in stoplist] for doc in doclist]

## Sentiment Analysis Cleaning
Beside the text, we also keep the VADER influence factors such as punctuations (exclamation marks) and capitulations.

In [0]:
def clean_text2(text):
    text  = text.replace('\n'," ")  #replce new line with space
    text = re.sub(r"@[a-zA-Z0-9_]{1,15}", "", text) #delete@username, @[any word, number, _]{length}
    text = re.sub(r"[a-zA-Z]*[:\//\]*[A-Za-z0-9\-_]+\.+[A-Za-z0-9\.\/%&=\?\-_]+", "", text)  #delete urls
    
    return text

In [0]:
docs_new = raw['text']
docs_new = docs_new.apply(lambda x :clean_text2(x))
texts_new = docs_new.values

# Most Frequent Words of Trump's Twitter

## Words Frequency

In [0]:
#calculate frequency of each words in texts
vocab_tf={}
for i in texts:
    for item in i:
        if item in vocab_tf:
            vocab_tf[item]+=1
        else:
            vocab_tf[item] = 1

In [0]:
vocab_tf_sorted = sorted(vocab_tf.items(), key=lambda x: x[1], reverse=True)
vocab_tf_sorted

[('great', 2390),
 ('president', 1318),
 ('people', 1216),
 ('trump', 1089),
 ('democrats', 914),
 ('country', 891),
 ('news', 870),
 ('thank', 861),
 ('big', 769),
 ('fake', 752),
 ('many', 699),
 ('border', 699),
 ('new', 688),
 ('today', 687),
 ('america', 684),
 ('get', 659),
 ('would', 644),
 ('time', 621),
 ('never', 578),
 ('much', 558),
 ('years', 546),
 ('media', 545),
 ('american', 539),
 ('good', 527),
 ('one', 520),
 ('want', 508),
 ('united', 495),
 ('states', 480),
 ('even', 476),
 ('make', 472),
 ('back', 469),
 ('china', 453),
 ('must', 444),
 ('done', 429),
 ('house', 421),
 ('going', 417),
 ('job', 412),
 ('vote', 410),
 ('state', 402),
 ('like', 401),
 ('jobs', 398),
 ('election', 390),
 ('us', 380),
 ('first', 373),
 ('bad', 369),
 ('dems', 369),
 ('military', 363),
 ('day', 361),
 ('made', 361),
 ('wall', 351),
 ('trade', 351),
 ('ever', 350),
 ('deal', 346),
 ('nothing', 336),
 ('crime', 334),
 ('win', 333),
 ('way', 332),
 ('security', 332),
 ('russia', 326),
 ('

## Word Cloud

In [0]:
wordcloud_string = ''
for i in range(775):
  temp = ' '.join(texts[i])
  wordcloud_string= ' '.join([wordcloud_string, temp])

In [0]:
w.generate(wordcloud_string)

In [0]:
#save the word cloud result
#w.to_file('/content/drive/My Drive/UR/DSC440 Data Mining/Project/Data/2.png')

# Topic Generation

## Corpora

In [0]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

## IF-IDF Model
By applying TF-IDF model to our corpora, we lower the weights of  meaningless words and raise the weights of true topic words in our corpora.

In [0]:
tfidf_model = models.TfidfModel(corpus)

In [0]:
corpus_tfidf = tfidf_model[corpus]

## LDA Model
We tried topic numbers from 10 to 50, selecting 20 as our best topic number.

We use corpus_tiidf as our LDA corpus, run 100 passes to stabilize our lda model.

In [0]:
# lda = gensim.models.ldamodel.LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=20, passes = 100)
#lda.save("/content/drive/My Drive/UR/DSC440 Data Mining/Project/Data/lda.model")
lda = gensim.models.ldamodel.LdaModel.load('/content/drive/My Drive/UR/DSC440 Data Mining/Project/Data/lda.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


I selected top 30 words from each topic, after eliminate meaningless words, extracted topic from those words.

In [0]:
# topic generation result
lda.show_topics(num_topics=20, num_words=30, log=False, formatted=False)

## Topic Distibution
Form our LDA model, we get each docoment's distribution over 20 topics

In [0]:
topic_distribution = []
for i in range(775):
  topic_distribution.append(lda.get_document_topics(corpus_tfidf[i], minimum_probability= 0.000001))

In [0]:
#pprint(topic_distribution)

# Sentiment Analysis

In [0]:
analyzer = SentimentIntensityAnalyzer()

In [0]:
sentiment_full_score = []
for i in range(775):
  sentiment_full_score.append(analyzer.polarity_scores(texts_new[i]))

In [0]:
# We extracted the compound score as a feature of our stock price to feed in our LSTM model in our next step.
sentiment_score = []
for i in range(775):
  normalized_data = (sentiment_full_score[i]['compound']+1)/2
  sentiment_score.append(normalized_data)

130/775 compound score are negative, which means Trump’s twitter’s sentiments are generally positive.

In [0]:
negative = 0
for i in range(775):
  if sentiment_full_score[i]['compound'] <0:
    negative += 1
negative

130

In [0]:
sentiment_full_score

[{'compound': -0.7941, 'neg': 0.12, 'neu': 0.821, 'pos': 0.059},
 {'compound': 0.9917, 'neg': 0.055, 'neu': 0.741, 'pos': 0.204},
 {'compound': 0.993, 'neg': 0.086, 'neu': 0.649, 'pos': 0.266},
 {'compound': 0.9766, 'neg': 0.033, 'neu': 0.737, 'pos': 0.23},
 {'compound': 0.9989, 'neg': 0.048, 'neu': 0.684, 'pos': 0.268},
 {'compound': 0.9963, 'neg': 0.032, 'neu': 0.703, 'pos': 0.265},
 {'compound': 0.9691, 'neg': 0.03, 'neu': 0.836, 'pos': 0.134},
 {'compound': 0.9893, 'neg': 0.065, 'neu': 0.64, 'pos': 0.295},
 {'compound': 0.9967, 'neg': 0.074, 'neu': 0.588, 'pos': 0.337},
 {'compound': 0.8976, 'neg': 0.0, 'neu': 0.775, 'pos': 0.225},
 {'compound': 0.2322, 'neg': 0.08, 'neu': 0.814, 'pos': 0.105},
 {'compound': 0.8271, 'neg': 0.0, 'neu': 0.7, 'pos': 0.3},
 {'compound': 0.8516, 'neg': 0.06, 'neu': 0.76, 'pos': 0.18},
 {'compound': 0.9862, 'neg': 0.075, 'neu': 0.728, 'pos': 0.197},
 {'compound': 0.9113, 'neg': 0.092, 'neu': 0.764, 'pos': 0.144},
 {'compound': 0.7089, 'neg': 0.18, 'neu':

# Final Result

In [0]:
raw_topic = pd.DataFrame(columns=['Topic1','Topic2','Topic3','Topic4','Topic5','Topic6','Topic7','Topic8','Topic9','Topic10','Topic11','Topic12','Topic13','Topic14','Topic15','Topic16','Topic17','Topic18','Topic19','Topic20'])

In [0]:
for i in range(775):
  distribution = []
  for j in range(20):
    distribution.append(topic_distribution[i][j][1])
  raw_topic.loc[i]=distribution

In [0]:
raw_topic['score'] = sentiment_score

In [0]:
raw_topic

,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,Topic20,score
0,0.007091,0.007091,0.007091,0.007091,0.393351,0.007091,0.007091,0.007091,0.007091,0.007091,0.007091,0.007091,0.007091,0.007091,0.479020,0.007091,0.007091,0.007091,0.007091,0.007091,0.10295
1,0.004474,0.004474,0.004474,0.004474,0.004474,0.004474,0.004474,0.004474,0.105289,0.004474,0.004474,0.312809,0.004474,0.004474,0.505849,0.004474,0.004474,0.004474,0.004474,0.004474,0.99585
2,0.005235,0.005235,0.005235,0.005235,0.005235,0.005235,0.005235,0.005235,0.122476,0.005235,0.005235,0.005235,0.247380,0.005235,0.541155,0.005235,0.005235,0.005235,0.005235,0.005235,0.99650
3,0.007046,0.007046,0.007046,0.007046,0.007046,0.360031,0.007046,0.007046,0.007046,0.007046,0.007046,0.007046,0.007046,0.007046,0.513145,0.007046,0.007046,0.007046,0.007046,0.007046,0.98830
4,0.004561,0.004561,0.004561,0.004561,0.004561,0.047843,0.004561,0.004561,0.065831,0.004561,0.004561,0.004561,0.004561,0.004561,0.525990,0.287353,0.004561,0.004561,0.004561,0.004561,0.99945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,0.003745,0.003745,0.003745,0.224122,0.003745,0.003745,0.003745,0.003745,0.123414,0.036808,0.003745,0.003745,0.003745,0.003745,0.555735,0.003745,0.003745,0.003745,0.003745,0.003745,0.97385
771,0.002458,0.026832,0.002458,0.002458,0.002458,0.018278,0.002458,0.002458,0.352546,0.002458,0.002458,0.020340,0.002458,0.002458,0.545131,0.002458,0.002458,0.002458,0.002458,0.002458,0.99970
772,0.003010,0.003010,0.003010,0.003010,0.003010,0.003010,0.003010,0.003010,0.117821,0.027999,0.003010,0.003010,0.003010,0.003010,0.575353,0.003010,0.003010,0.003010,0.003010,0.230664,0.01190
773,0.003497,0.003497,0.003497,0.003497,0.003497,0.003497,0.003497,0.003497,0.095284,0.003497,0.003497,0.003497,0.003497,0.003497,0.543884,0.301381,0.003497,0.003497,0.003497,0.003497,0.98815


In [0]:
raw = pd.concat([raw, raw_topic], axis=1)
raw

,date,text,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,Topic20,score
0,2016-11-01,So terrible that Crooked didn't report she got...,0.007091,0.007091,0.007091,0.007091,0.393351,0.007091,0.007091,0.007091,0.007091,0.007091,0.007091,0.007091,0.007091,0.007091,0.479020,0.007091,0.007091,0.007091,0.007091,0.007091,0.10295
1,2016-11-02,Praying for the families of the two Iowa polic...,0.004474,0.004474,0.004474,0.004474,0.004474,0.004474,0.004474,0.004474,0.105289,0.004474,0.004474,0.312809,0.004474,0.004474,0.505849,0.004474,0.004474,0.004474,0.004474,0.004474,0.99585
2,2016-11-03,Looking at Air Force One @ MIA. Why is he camp...,0.005235,0.005235,0.005235,0.005235,0.005235,0.005235,0.005235,0.005235,0.122476,0.005235,0.005235,0.005235,0.247380,0.005235,0.541155,0.005235,0.005235,0.005235,0.005235,0.005235,0.99650
3,2016-11-04,"Join me today in Wilmington, Ohio at 4pm: http...",0.007046,0.007046,0.007046,0.007046,0.007046,0.360031,0.007046,0.007046,0.007046,0.007046,0.007046,0.007046,0.007046,0.007046,0.513145,0.007046,0.007046,0.007046,0.007046,0.007046,0.98830
4,2016-11-07,Thank you for you support Virginia! In ONE DAY...,0.004561,0.004561,0.004561,0.004561,0.004561,0.047843,0.004561,0.004561,0.065831,0.004561,0.004561,0.004561,0.004561,0.004561,0.525990,0.287353,0.004561,0.004561,0.004561,0.004561,0.99945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,2019-10-25,“Donald J. Trump is an absolutely historic Pre...,0.003745,0.003745,0.003745,0.224122,0.003745,0.003745,0.003745,0.003745,0.123414,0.036808,0.003745,0.003745,0.003745,0.003745,0.555735,0.003745,0.003745,0.003745,0.003745,0.003745,0.97385
771,2019-10-28,The S&amp;P just hit an ALL TIME HIGH. This is...,0.002458,0.026832,0.002458,0.002458,0.002458,0.018278,0.002458,0.002458,0.352546,0.002458,0.002458,0.020340,0.002458,0.002458,0.545131,0.002458,0.002458,0.002458,0.002458,0.002458,0.99970
772,2019-10-29,Nervous Nancy Pelosi is doing everything possi...,0.003010,0.003010,0.003010,0.003010,0.003010,0.003010,0.003010,0.003010,0.117821,0.027999,0.003010,0.003010,0.003010,0.003010,0.575353,0.003010,0.003010,0.003010,0.003010,0.230664,0.01190
773,2019-10-30,....the United States Military. Congress needs...,0.003497,0.003497,0.003497,0.003497,0.003497,0.003497,0.003497,0.003497,0.095284,0.003497,0.003497,0.003497,0.003497,0.003497,0.543884,0.301381,0.003497,0.003497,0.003497,0.003497,0.98815


The final result is saved as a tweets_score.csv file

In [0]:
#raw.to_csv("/content/drive/My Drive/UR/DSC440 Data Mining/Project/Data/tweets_score.csv", columns=['date','text','Topic1','Topic2','Topic3','Topic4','Topic5','Topic6','Topic7','Topic8','Topic9','Topic10','Topic11','Topic12','Topic13','Topic14','Topic15','Topic16','Topic17','Topic18','Topic19','Topic20','score'], index=0, header=1)